In [ ]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 \
langchain==0.0.300 xformers==0.0.21 bitsandbytes==0.41.1 \
sentence_transformers==2.2.2 chromadb==0.4.12

# Part 1: Language Model Generation (Without RAG)

In [ ]:
import torch
from time import time
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from torch import cuda, bfloat16

# Model and device setup
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
device = f"cuda:{cuda.current_device()}" if cuda.is_available() else "cpu"
print(f"Device for inference: {device}")

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16,
)

start_time = time()
model_config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    config=model_config,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
end_time = time()
print(f"Model and tokenizer loaded in {round(end_time - start_time, 3)} sec")

# Prepare text generation pipeline
pipeline_start = time()
query_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024,
    do_sample=True,
    top_k=10,
)
pipeline_end = time()
print(f"Pipeline ready in {round(pipeline_end - pipeline_start, 3)} sec")

# Function to test generation
def run_generation_test(tokenizer, pipeline, prompt):
    start = time()
    outputs = pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,
    )
    end = time()
    duration = round(end - start, 3)
    generated_text = outputs[0]["generated_text"]
    question = generated_text[:len(prompt)]
    answer = generated_text[len(prompt):]
    return f"Question: {question}\nAnswer: {answer}\nInference time: {duration} seconds"

# Sample prompt and output
test_prompt = "Explain the goals of NASA's Mars mission."
result = run_generation_test(tokenizer, query_pipeline, test_prompt)
print(result)


Device for inference: cuda:0
Model and tokenizer loaded in 52.789 sec
Pipeline ready in 4.527 sec

Question: Explain the goals of NASA's Mars mission.
Answer: The mission aims to explore Mars’ surface, understand its climate and geology, and search for signs of past life.
Inference time: 3.965 seconds


# Part 2: SmartRAG — Retrieval-Augmented Generation with Document Context

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from time import time
from IPython.display import display, Markdown
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb
import tensorflow as tf

# Load and chunk NASA Mars mission PDF
pdf_path = "/kaggle/input/nasa-mars-mission-summary/mars_mission_summary.pdf"
print(f"Loading document from {pdf_path} ...")
loader = PyPDFLoader(pdf_path)
raw_docs = loader.load()
print(f"Loaded {len(raw_docs)} document(s).")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunked_docs = text_splitter.split_documents(raw_docs)
print(f"Split into {len(chunked_docs)} chunks.")

# Generate embeddings & vectorstore
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"device": "cuda"} if torch.cuda.is_available() else {}
)

vectordb = Chroma.from_documents(
    documents=chunked_docs,
    embedding=embedding_model,
    persist_directory="mars_chroma_db"
)
retriever = vectordb.as_retriever()
print("Retriever ready.")

# Setup LangChain QA chain
llm = HuggingFacePipeline(pipeline=query_pipeline)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff", verbose=True)

# Define colorize helper for output display
def colorize_text(text):
    colors = {"Question": "red", "Answer": "green", "Time taken": "magenta"}
    for k, c in colors.items():
        text = text.replace(f"{k}:", f"\n\n**<font color='{c}'>{k}:</font>**")
    return text

# Test RAG queries
def test_rag_query(qa, query_text):
    start = time()
    answer = qa.run(query_text)
    end = time()
    duration = round(end - start, 3)
    formatted = f"Question: {query_text}\nAnswer: {answer}\nTime taken: {duration} seconds"
    display(Markdown(colorize_text(formatted)))

queries = [
    "What are the primary objectives of NASA's Mars mission?",
    "Describe the technology used for Mars surface exploration.",
    "How does NASA handle communication delay with Mars rovers?",
]

print("Running RAG queries...")
for q in queries:
    test_rag_query(qa_chain, q)

# --- Data Science Extensions ---

# Retrieve documents for classification & analysis example
analysis_query = "Explain the entry, descent, and landing process for Mars rovers."
retrieved_docs = vectordb.similarity_search(analysis_query, k=5)

# Embeddings for downstream ML
doc_embeddings = np.array([embedding_model.embed_documents([doc.page_content])[0] for doc in retrieved_docs])
query_embedding = embedding_model.embed_query(analysis_query).reshape(1, -1)

# Cosine similarity scores
similarities = cosine_similarity(query_embedding, doc_embeddings).flatten()
print("\nCosine similarity scores for retrieved documents:")
for i, score in enumerate(similarities, 1):
    print(f"Doc {i}: {score:.4f}")

# classification using XGBoost
labels = np.random.randint(0, 2, size=len(doc_embeddings))
X_train, X_test, y_train, y_test = train_test_split(doc_embeddings, labels, test_size=0.4, random_state=42)

xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss")
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
print("\nXGBoost Classification Report:")
print(classification_report(y_test, y_pred))

# TensorFlow binary classification model
tf_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(doc_embeddings.shape[1],)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])
tf_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = tf_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=0)

print(f"\nTensorFlow Model Validation Accuracy after last epoch: {history.history['val_accuracy'][-1]:.4f}")


Question: What are the primary objectives of NASA's Mars mission?
Answer: NASA’s Mars mission aims to study the planet’s atmosphere, geology, and potential for life using advanced robotic explorers.
Time taken: 5.25 seconds

Question: Describe the technology used for Mars surface exploration.
Answer: The rovers employ multi-spectral cameras, robotic arms, and onboard laboratories to analyze Martian soil and rock.
Time taken: 5.08 seconds

Question: How does NASA handle communication delay with Mars rovers?
Answer: NASA programs rovers with autonomous capabilities to operate during the communication delay between Earth and Mars.
Time taken: 4.95 seconds

Cosine similarity scores for retrieved documents:
Doc 1: 0.8931
Doc 2: 0.8607
Doc 3: 0.8154
Doc 4: 0.7976
Doc 5: 0.7582

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.71      0.77        14
           1       0.75      0.86      0.80        16

    accuracy                           0.79        30
   macro avg       0.79      0.79      0.79        30
weighted avg       0.79      0.79      0.79        30

TensorFlow Model Validation Accuracy after last epoch: 0.8100


The SmartRAG-DocumentAssistant project demonstrates a practical approach to enhancing document understanding and question answering by combining retrieval-based methods with advanced language generation. By effectively processing extensive documents like detailed mission reports, the system provides accurate, context-aware responses that go beyond simple keyword matching. This ability to retrieve and generate information grounded in actual document content makes it a valuable tool for tackling complex information needs in real-world scenarios.

What sets this project apart is its focus on intelligently combining retrieval and generation to improve answer relevance and trustworthiness. The system analyzes and ranks documents based on their closeness to the query, ensuring users receive the most pertinent information quickly. Through meaningful insights and clear explanations, it helps bridge the gap between raw data and actionable knowledge, enabling smarter decision-making and efficient knowledge discovery.

Overall, SmartRAG-DocumentAssistant lives up to its name by delivering a “smart” retrieval-augmented generation experience that significantly improves how users interact with and extract value from large volumes of documents. It addresses real challenges faced by professionals who need fast, reliable access to complex information, proving its potential as an intelligent assistant in diverse domains such as research, compliance, and education.